In [1]:
siteUrl = "http://www.autotrader.com/cars-for-sale/Bellevue+NE-68147"
siteUrl = "?".join([siteUrl, "zip=68147"])
siteUrl = "&".join([siteUrl, "sellerTypes=p"])
siteUrl = "&".join([siteUrl, "showcaseOwnerId=0"])
siteUrl = "&".join([siteUrl, "startYear=1981"])
siteUrl = "&".join([siteUrl, "endYear=2018"])
siteUrl = "&".join([siteUrl, "searchRadius=0"])
siteUrl = "&".join([siteUrl, "numRecords=100"])
print(siteUrl)

proxies_array = [
    {'host': '64.120.85.192', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Chicago Ilinois'},
    {'host': '8.29.125.84', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Columbus Ohio'},
    {'host': '173.234.204.209', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Dallas Texas'},
    {'host': '8.29.124.215', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Denver Colorado'},
    {'host': '167.160.106.72', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Houston Texas'},
    {'host': '192.171.229.113', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Irvine California'},
    {'host': '45.59.21.72', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Las Vegas Nevada'},
    {'host': '142.91.235.188', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Los Angeles California'},
    {'host': '108.62.137.20', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'New York City New York'},
    {'host': '216.107.136.72', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Oklahoma City Oklahoma'},
    {'host': '23.19.213.86', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Phoenix Arizona'},
]

http://www.autotrader.com/cars-for-sale/Bellevue+NE-68147?zip=68147&sellerTypes=p&showcaseOwnerId=0&startYear=1981&endYear=2018&searchRadius=0&numRecords=100


In [2]:
from random import randint

proxy = proxies_array[randint(0, len(proxies_array)-1)]
print(proxy)

{'loc': 'Los Angeles California', 'pwd': '<<PASSWORD>>', 'port': '56597', 'host': '142.91.235.188', 'usr': '<<USERID>>'}


In [3]:
from selenium import webdriver
from base64 import b64encode

def get_firefox_credentials(proxy):
    credentials = '{usr}:{pwd}'.format(**proxy)
    credentials = b64encode(credentials.encode('ascii')).decode('utf-8')
    
    return(credentials)

def get_firefox_profile(proxy, credentials):
    fp = webdriver.FirefoxProfile()
    fp.set_preference('network.proxy.type', 1)
    fp.set_preference('network.proxy.http', proxy['host'])
    fp.set_preference('network.proxy.http_port', int(proxy['port']))
    fp.set_preference('network.proxy.no_proxies_on', 'localhost, 127.0.0.1')
    fp.add_extension('closeproxy.xpi')

    # Set up the credentials
    fp.set_preference('extensions.closeproxyauth.authtoken', credentials)

    #set some privacy settings
    fp.set_preference( "places.history.enabled", False )
    fp.set_preference( "privacy.clearOnShutdown.offlineApps", True )
    fp.set_preference( "privacy.clearOnShutdown.passwords", True )
    fp.set_preference( "privacy.clearOnShutdown.siteSettings", True )
    fp.set_preference( "privacy.sanitize.sanitizeOnShutdown", True )
    fp.set_preference( "signon.rememberSignons", False )
    fp.set_preference( "network.cookie.lifetimePolicy", 2 )
    fp.set_preference( "network.dns.disablePrefetch", True )
    fp.set_preference( "network.http.sendRefererHeader", 0 )

    #if you're really hardcore about your security
    #js can be used to reveal your true i.p.
    #fp.set_preference( "javascript.enabled", False )

    #get a huge speed increase by not downloading images
    #fp.set_preference( "permissions.default.image", 2 )
    
    return(fp)

proxy = proxies_array[randint(0, len(proxies_array)-1)]
credentials = get_firefox_credentials(proxy)
fp = get_firefox_profile(proxy, credentials)

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import zipfile

def get_chrome_options(proxy):
    manifest_json = """
    {
        "version": "1.0.0",
        "manifest_version": 2,
        "name": "Chrome Proxy",
        "permissions": [
            "proxy",
            "tabs",
            "unlimitedStorage",
            "storage",
            "<all_urls>",
            "webRequest",
            "webRequestBlocking"
        ],
        "background": {
            "scripts": ["background.js"]
        },
        "minimum_chrome_version":"22.0.0"
    }
    """

    background_js = """
    var config = {{
            mode: "fixed_servers",
            rules: {{
              singleProxy: {{
                scheme: "http",
                host: "{host}",
                port: parseInt({port})
              }},
              bypassList: ["foobar.com"]
            }}
          }};

    chrome.proxy.settings.set({{value: config, scope: "regular"}}, function() {{}});

    function callbackFn(details) {{
        return {{
            authCredentials: {{
                username: "{usr}",
                password: "{pwd}"
            }}
        }};
    }}

    chrome.webRequest.onAuthRequired.addListener(
                callbackFn,
                {{urls: ["<all_urls>"]}},
                ['blocking']
    );
    """.format(**proxy)


    pluginfile = 'proxy_auth_plugin.zip'

    with zipfile.ZipFile(pluginfile, 'w') as zp:
        zp.writestr("manifest.json", manifest_json)
        zp.writestr("background.js", background_js)

    co = Options()
    co.add_argument("--start-maximized")
    co.add_argument('--dns-prefetch-disable')
    #co.add_argument('--disable-extensions')
    co.add_extension(pluginfile)
    
    return(co)

In [5]:
print(background_js)


var config = {
        mode: "fixed_servers",
        rules: {
          singleProxy: {
            scheme: "http",
            host: "167.160.106.72",
            port: parseInt(56597)
          },
          bypassList: ["foobar.com"]
        }
      };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "<<USERID>>",
            password: "<<PASSWORD>>"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);



In [6]:
co = get_chrome_options(proxy)
driver = webdriver.Chrome(chrome_options=co)

#driver = webdriver.Firefox(fp)

In [7]:
import time
from bs4 import BeautifulSoup

# set timeout information
driver.set_page_load_timeout(50)

finished = 0
fails = 0
while finished == 0 and fails < 8:
    try:
        driver.get(siteUrl)
        finished = 1
    except Exception as e:
        print(str(e))
        fails = fails + 1
        time.sleep(5)

siteSoup = BeautifulSoup(driver.page_source, 'lxml')

In [8]:
import re

#<strong data-qaid="cntnr-resultTotal" data-reactid="133">76,789</strong>
resultCss = 'strong[data-qaid="cntnr-resultTotal"]'
resultTag = siteSoup.select(resultCss)
resultText = ""
if len(resultTag):
    resultText = resultTag[0].get_text().strip()

# Get the max firstRecord
max_firstRecord = 0
if len(resultText):
    max_firstRecord = min(900, int(re.sub(r'\D+', '', resultText)))
print(max_firstRecord)

900


In [9]:
from random import randint
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

page_array = list(range(0, max_firstRecord+1, 100))
i = page_array[randint(0,len(page_array)-1)]
tagUrl = "&".join([siteUrl, "firstRecord="+str(i)])
print(tagUrl)

finished = 0
fails = 0
while finished == 0 and fails < 8:
    try:
        driver.get(tagUrl)
        finished = 1
    except Exception as e:
        print(str(e))
        fails = fails + 1
        time.sleep(5)

# Get the title of the page to prove we are progressing
try:
    
    # Wait for title to show up
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "head > title"))
    )
    
except Exception as e:
    print(str(e))

soup = BeautifulSoup(driver.page_source, 'lxml')
tagTitle = soup.select('head > title')
titleString = tagTitle[0].text.strip()
print(titleString)

http://www.autotrader.com/cars-for-sale/Bellevue+NE-68147?zip=68147&sellerTypes=p&showcaseOwnerId=0&startYear=1981&endYear=2018&searchRadius=0&numRecords=100&firstRecord=700



In [15]:
import urllib.parse

listingSoup = BeautifulSoup(driver.page_source, 'lxml')

#<li data-qaid="cntnr-lsting-ownername" title="Private Seller" style="overflow:hidden;text-overflow:ellipsis;" 
#data-reactid="1163">Private Seller</li>
privateTags = listingSoup.select('li[title="Private Seller"]')

privateTag = privateTags[randint(0,len(privateTags)-1)]
pageSoup = privateTag.parent.parent.parent.parent

#<a href="/cars-for-sale/vehicledetails.xhtml?listingId=434999951&amp;zip=68147&amp;
#referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D68147%26sellerTypes%3Dp%26showcaseOwnerId%3D0%26startYear
#%3D1981%26numRecords%3D100%26firstRecord%3D800%26sellerType%3Dp%26endYear%3D2018%26searchRadius%3D0&amp;
#sellerTypes=p&amp;showcaseOwnerId=0&amp;startYear=1981&amp;numRecords=100&amp;firstRecord=800&amp;endYear=2018&amp;
#searchRadius=0&amp;makeCode1=JAG&amp;modelCode1=JAGFTYPE" data-qaid="lnk-lstgTtlf" data-birf-clickdelay="false" 
#title="Used 2015 Jaguar F-TYPE R" class="text-md" data-birf-log="component" data-birf-cmp="v_lstg_t" 
#data-birf-par="v_lstg_2" data-birf-extra="{&quot;v&quot;:434999951,&quot;c&quot;:70672389,&quot;
#lstg_type&quot;:&quot;r&quot;,&quot;z&quot;:&quot;68147&quot;,&quot;spt&quot;:&quot;n&quot;}" data-reactid="1118">
#<strong style="font-size:16px;" data-reactid="1119">Used 2015 Jaguar F-TYPE</strong><!-- react-text: 1120 --> 
#<!-- /react-text --><!-- react-text: 1121 -->R<!-- /react-text --></a>
pageLinks = pageSoup.select('a[data-qaid="lnk-lstgTtlf"]')
#print(pageLinks)

pageUrl = urllib.parse.urljoin(tagUrl, re.sub(' ', '%20', pageLinks[0]['href']))
print(pageUrl)

http://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=448806308&zip=68147&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D68147%26sellerTypes%3Dp%26showcaseOwnerId%3D0%26startYear%3D1981%26numRecords%3D100%26incremental%3Dall%26firstRecord%3D700%26endYear%3D2018%26searchRadius%3D0&sellerTypes=p&startYear=1981&numRecords=100&firstRecord=700&endYear=2018&searchRadius=0&makeCode1=ASTON&modelCode1=DB9


In [16]:
import urllib.parse

page_array = list(range(0, max_firstRecord+1, 100))
seen_urls_array = []
for i in page_array:
    tagUrl = "&".join([siteUrl, "firstRecord="+str(i)])

    finished = 0
    fails = 0
    while finished == 0 and fails < 8:
        try:
            driver.get(tagUrl)
            finished = 1
        except Exception as e:
            print(str(e))
            fails = fails + 1
            time.sleep(5)
    
    listingSoup = BeautifulSoup(driver.page_source, 'lxml')

    privateTags = listingSoup.select('li[title="Private Seller"]')
    for privateTag in privateTags:
        pageSoup = privateTag.parent.parent.parent.parent

        # Get the links to the individual pages
        pageLinks = pageSoup.select('a[data-qaid="lnk-lstgTtlf"]')
        for pageLink in pageLinks:
            pageUrl = urllib.parse.urljoin(tagUrl, re.sub(' ', '%20', pageLink['href']))
            if pageUrl not in seen_urls_array:
                seen_urls_array.append(pageUrl)

        # Destroy the tree when you're done working with it
        pageSoup.decompose()
    
    # Destroy the tree when you're done working with it
    listingSoup.decompose()

# Show the number of individual page links you have
print(len(seen_urls_array))

1000


In [17]:
import pickle

with open('autotrader.pickle', 'wb') as handle:
    pickle.dump(seen_urls_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
import pickle

with open('autotrader.pickle', 'rb') as handle:
    seen_urls_array = pickle.load(handle)

In [19]:
from random import randint

pageUrl = seen_urls_array[randint(0,len(seen_urls_array)-1)]
print(pageUrl)

http://www.autotrader.com/cars-for-sale/vehicledetails.xhtml?listingId=397221994&zip=68147&referrer=%2Fcars-for-sale%2Fsearchresults.xhtml%3Fzip%3D68147%26sellerTypes%3Dp%26showcaseOwnerId%3D0%26startYear%3D1981%26numRecords%3D100%26incremental%3Dall%26firstRecord%3D100%26endYear%3D2018%26searchRadius%3D0&sellerTypes=p&startYear=1981&numRecords=100&firstRecord=100&endYear=2018&searchRadius=0&makeCode1=BENTL&modelCode1=BENCONT


In [20]:
finished = 0
fails = 0
while finished == 0 and fails < 8:
    try:
        driver.get(pageUrl)
        finished = 1
    except Exception as e:
        print(str(e))
        fails = fails + 1
        time.sleep(5)

In [21]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    #<div class="text-link" data-qaid="dlr_phone_lnk" data-birf-log="component" data-birf-cmp="gp" data-birf-par="" 
    #data-birf-extra="{&quot;c&quot;:70939793,&quot;v&quot;:448319007,&quot;novs&quot;:&quot;n&quot;}" 
    #data-reactid="205">Get this seller's phone number</div>
    buttonCss = 'div[data-qaid="dlr_phone_lnk"]'
    
    # Wait for button to show up
    buttonTag = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, buttonCss))
    )
    
    buttonTag.click()
except Exception as e:
    print(str(e))
#driver.save_screenshot('cycletrader.png')

In [14]:
driver.quit()

In [22]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

phoneRe = re.compile(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}")

#<a data-qaid="dlr_phone" href="tel:305-775-7140">305-775-7140</a>
phoneCss = 'a[data-qaid="dlr_phone"]'

# Wait for button to disappear
#print(driver.find_element_by_css_selector(buttonCss).get_attribute('outerHTML'))
WebDriverWait(driver, 30).until_not(
    EC.visibility_of_element_located((By.CSS_SELECTOR, buttonCss))
)

phoneHTML = driver.find_element_by_css_selector(phoneCss).get_attribute('innerHTML')
#print(phoneHTML)
match = phoneRe.search(phoneHTML)
if match:
    phone = match.group()
else:
    phone = ""
print(phone)

973-277-7825


In [23]:
try:
    
    #<span data-qaid="cntnr-vehicle-title" data-reactid="163">Used 2016 Jaguar F-TYPE</span>
    model = driver.find_element_by_css_selector('span[data-qaid="cntnr-vehicle-title"]').text.strip()
    if len(model):
        model = re.sub(r'\s+', ' ', model)
    else:
        model = "Unknown"
    print(model)
    
    #<strong class="text-lg" data-qaid="cntnr-lstng-price1" data-reactid="187">$77,500</strong>
    price = driver.find_element_by_css_selector('strong[data-qaid="cntnr-lstng-price1"]').text.strip()
    if not len(price):
        price = "Unknown"
    print(price)

except Exception as e:
    print(str(e))
finally:
    driver.quit()

Used 2012 Bentley Continental
$155,000


In [25]:
import random
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

phoneRe = re.compile(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}")
buttonCss = 'div[data-qaid="dlr_phone_lnk"]'
phoneCss = 'a[data-qaid="dlr_phone"]'
modelCss = 'span[data-qaid="cntnr-vehicle-title"]'
priceCss = 'strong[data-qaid="cntnr-lstng-price1"]'
scrambled_urls_array = sorted(seen_urls_array, key = lambda x: random.random())
for pageUrl in scrambled_urls_array:
    try:
        proxy = proxies_array[randint(0, len(proxies_array)-1)]
        co = get_chrome_options(proxy)
        driver = webdriver.Chrome(chrome_options=co)

        # Set timeout information
        driver.set_page_load_timeout(50)

        # Bring up the page
        finished = 0
        fails = 0
        while finished == 0 and fails < 8:
            try:
                driver.get(pageUrl)
                finished = 1
            except Exception as e:
                print(str(e))
                fails = fails + 1
                time.sleep(5)

        # Click the phone button
        try:

            # Wait for button to show up
            buttonTag = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, buttonCss))
            )

            buttonTag.click()
        except Exception as e:
            print(str(e))

        # Get the phone number

        # Wait for button to disappear
        WebDriverWait(driver, 30).until_not(
            EC.visibility_of_element_located((By.CSS_SELECTOR, buttonCss))
        )

        phoneHTML = driver.find_element_by_css_selector(phoneCss).get_attribute('innerHTML')
        match = phoneRe.search(phoneHTML)
        if match:
            phone = match.group()
        else:
            phone = ""
        #print(phone)

        if len(phone):

            model = driver.find_element_by_css_selector(modelCss).text.strip()
            if len(model):
                model = re.sub(r'\s+', ' ', model)
            else:
                model = "Unknown"
            #print(model)

            price = driver.find_element_by_css_selector(priceCss).text.strip()
            if not len(price):
                price = "Unknown"
            #print(price)

            bigFile = open('autotrader.txt', 'a', encoding='utf-8')
            bigFile.write(model + '\t' + price + '\t' + phone + '\n')
            bigFile.close()

        else:
            print(phoneHTML)
            driver.save_screenshot('autotrader.png')

    except Exception as e:
        print(str(e))
    finally:
        driver.quit()

Message: timeout: cannot determine loading status
from timeout: Timed out receiving message from renderer: -0.038
  (Session info: chrome=56.0.2924.87)
  (Driver info: chromedriver=2.27.440174 (e97a722caafc2d3a8b807ee115bfb307f7d2cfd9),platform=Windows NT 10.0.14393 x86_64)

Message: timeout: cannot determine loading status
from timeout: Timed out receiving message from renderer: -0.060
  (Session info: chrome=56.0.2924.87)
  (Driver info: chromedriver=2.27.440174 (e97a722caafc2d3a8b807ee115bfb307f7d2cfd9),platform=Windows NT 10.0.14393 x86_64)

Message: timeout: cannot determine loading status
from timeout: Timed out receiving message from renderer: -0.029
  (Session info: chrome=56.0.2924.87)
  (Driver info: chromedriver=2.27.440174 (e97a722caafc2d3a8b807ee115bfb307f7d2cfd9),platform=Windows NT 10.0.14393 x86_64)

Message: timeout: cannot determine loading status
from timeout: Timed out receiving message from renderer: -0.041
  (Session info: chrome=56.0.2924.87)
  (Driver info: chro